<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse from LightGBM

### train, val, test data로 분류

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import datetime
import os, re

In [2]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm
import joblib

In [3]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

In [4]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import make_scorer

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [5]:
import pickle

# write list, dictionary to pickle
def save_to_pickle(path, filename):
    open_file = open(path, "wb")
    pickle.dump(filename, open_file)
    open_file.close()

# read list, dictionary from pickle
def load_from_pickle(path):
    open_file = open(path, "rb")
    loaded_file = pickle.load(open_file)
    open_file.close()
    return loaded_file

In [6]:
import csv

# write list, dictionary to csv
# path = './xxx/', my_dict = filename

def save_dict_to_csv(path, my_dict):
    fields=my_dict.keys()
    values = my_dict.values()
    pd.DataFrame([fields, values], index=['parameter','value']).T.to_csv(path)
#     df = pd.DataFrame.from_dict(my_dict, orient='index') 
#     df.to_csv (path, index=False, header=True)  
    
def save_list_to_csv(path, my_list):
    df = pd.DataFrame(my_list, columns=['columns'])
    df.to_csv (path, index=False, header=True) 
    
# def load_dict_from_csv(path):
#     df = pd.read_csv(path, header=None)
#     my_dict = df.to_dict()
#     return my_dict

In [7]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]
    return collect_list

In [8]:
def make_df_from_estimator(estimator, num):
    df_t = pd.DataFrame.from_dict(estimator, orient='index')
    df_t.columns = [f'iter_{num}']
    df_t.index.name = 'parameter'
    return df_t

In [9]:
def calc_results(lgbm, model, train_scaled, val_scaled, test_scaled, train_target, val_target, test_target):
# model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
    y_predict_train = model.predict(train_scaled)
    y_predict_val = model.predict(val_scaled)
    y_predict_test = model.predict(test_scaled)
    result_dict= {}
    result_dict['best_score'] = lgbm.best_score_ 
    result_dict['best_index'] = lgbm.best_index_
    result_dict['acc_train'] = model.score(train_scaled, train_target)
    result_dict['acc_val'] = model.score(val_scaled, val_target)
    result_dict['acc_test'] = model.score(test_scaled, test_target)
    result_dict['train_precision'] = precision_score(train_target, y_predict_train)
    cm = confusion_matrix(train_target, y_predict_train)
    result_dict['train_tn'] = cm[0,0]
    result_dict['train_fp'] = cm[0,1]
    result_dict['train_fn'] = cm[1,0]
    result_dict['train_tp'] = cm[1,1]
    result_dict['val_precision'] = precision_score(val_target, y_predict_val)
    cm = confusion_matrix(val_target, y_predict_val)
    result_dict['val_tn'] = cm[0,0]
    result_dict['val_fp'] = cm[0,1]
    result_dict['val_fn'] = cm[1,0]
    result_dict['val_tp'] = cm[1,1]
    result_dict['test_precision'] = precision_score(test_target, y_predict_test)
    result_dict['recall'] = recall_score(test_target, y_predict_test)
    result_dict['f1score'] = f1_score(test_target, y_predict_test)
    result_dict['roc'] = roc_auc_score(test_target, y_predict_test)
    cm = confusion_matrix(test_target, y_predict_test)
    result_dict['test_tn'] = cm[0,0]
    result_dict['test_fp'] = cm[0,1]
    result_dict['test_fn'] = cm[1,0]
    result_dict['test_tp'] = cm[1,1]
#     result_dict['precision_neg'] = cm[0,0] / (cm[0,0] + cm[1,0])
    return result_dict

In [10]:
def save_best_results(com_name, model, scaler, columns, df_base):
    joblib.dump(model, f'{directory_for_model}{com_name}/best_model.pkl') # estimaor 저장
    joblib.dump(scaler, f'{directory_for_model}{com_name}/best_scaler.pkl') # scaler 저장
    save_to_pickle(f'{directory_for_model}{com_name}/best_model_p.pkl', model)
    save_to_pickle(f'{directory_for_model}{com_name}/best_scaler_p.pkl', scaler)
    save_to_pickle(f'{directory_for_model}{com_name}/best_columns.pkl', new_col)
    save_list_to_csv(f'{directory_for_model}{com_name}/best_columns.csv', new_col)
    df_base.to_pickle(f'{directory_for_model}{com_name}/best_result.pkl')
    return

In [11]:
def add_dict(*dicts):
    dsum = {}
    d_k = []
    d_v = []
    for dic in dicts:
        d_k.extend(list(dic.keys()))  # sum keys list
        d_v.extend(list(dic.values()))  # sum values list
    for i in range(len(d_k)):
        dsum[d_k[i]] = d_v[i]
    return dsum

In [12]:
def df_combine_sorted(df_base_l, df_params_l):
    o_num = list(df_base_l.index).index('best_score')
    df1 = df_base.iloc[:o_num, :]
    df2 = df_base.iloc[o_num:, :]
    df3 = pd.concat([df1, df_params_l], axis=0)
    df3.sort_index(axis=0, inplace=True)
    df_sorted = pd.concat([df3, df2], axis=0)
    return df_sorted

In [13]:
import sys, os

module_path = os.path.abspath(os.path.join('../..')) # 현재 폴더로 이동
if module_path+"\\data\\base_data\\common_data" not in sys.path:
    sys.path.append(module_path+"\\data\\base_data\\common_data") #  공통으로 사용하는 각종 리스트, 코드 등 
    
import common_data as cd

In [14]:
def df_style(df_s, row_index_l):
#     xin = [list(df_s.index).index(x) for x in row_index_l]
    xin = []
    for x in list(df_s.index):
        try:
            if x in row_index_l:
                xin.append(list(df_s.index).index(x))
        except:
            pass
    return df_s.reset_index('parameter').style.apply(
        lambda x: ['background-color: yellow' if row in xin else '' for row in range(len(df_s.index))], axis=0)

In [15]:
code = cd.code_all # 전체 회사 코드
code =  {'003490': ['대한항공', 'koreanair']}

In [16]:
col_inv1 = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1', 'pension_1', 
#             'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1', 'corporateetc_1', 
            'privequity_1',  'insurance_1', 'corporateetc_1', # bank_1, 'financeetc_1 제외
            'foreigneretc_1']
col_inv2 = ['retail_2', 'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2', 'pension_2',
#             'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2', 'corporateetc_2', 
            'privequity_2', 'insurance_2', 'corporateetc_2', # bank_2, 'financeetc_2 제외
            'foreigneretc_2']
col_his1 = ['open_1', 'high_1', 'low_1', 'close_1', 'vol_1']
col_his2 = ['open_2', 'high_2', 'low_2', 'close_2', 'vol_2']
col_cr = ['weekday', 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']
col_common1 = ["dji_cr", "dji_f_cr", "dxy_cr", "ixic_f_cr", "bond_kor_10_cr", "bond_kor_2_cr", "kosdaq_cr", "kospi_cr", 
         "krw_cr", "ixic_cr", "spx_f_cr", "sox_cr", "spx_cr", "bond_usa_10_cr", "bond_usa_2_cr", "bond_usa_3m_cr", 
         "vix_cr", "wti_cr", "spsy_cr", "spny_cr", "spxhc_cr", "splrcd_cr", "splrci_cr", "splrcu_cr", "splrcs_cr",
         "splrct_cr", "splrcl_cr", "splrcm_cr", "ixbk_cr", "ixfn_cr", "ixid_cr", "ixis_cr", "ixk_cr", "ixtr_cr",
         "ixut_cr", "nbi_cr", "bkx_cr"]
col_common2 = ["dji_cr_2", "dji_f_cr_2", "dxy_cr_2", "ixic_f_cr_2", "bond_kor_10_cr_2", "bond_kor_2_cr_2", "kosdaq_cr_2", "kospi_cr_2",
         "krw_cr_2", "ixic_cr_2", "spx_f_cr_2", "sox_cr_2", "spx_cr_2", "bond_usa_10_cr_2", "bond_usa_2_cr_2", "bond_usa_3m_cr_2",
         "vix_cr_2", "wti_cr_2", "spsy_cr_2", "spny_cr_2", "spxhc_cr_2", "splrcd_cr_2", "splrci_cr_2", "splrcu_cr_2",
         "splrcs_cr_2", "splrct_cr_2", "splrcl_cr_2", "splrcm_cr_2", "ixbk_cr_2", "ixfn_cr_2", "ixid_cr_2",
         "ixis_cr_2", "ixk_cr_2", "ixtr_cr_2", "ixut_cr_2", "nbi_cr_2", "bkx_cr_2"]
col_futures = ['ixic_f_cr', 'ixic_f_cr_2', 'spx_f_cr', 'spx_f_cr_2', 'dji_f_cr', 'dji_f_cr_2',
           'wti_cr','wti_cr_2', 'dxy_cr', 'dxy_cr_2', 'bond_usa_10_cr', 'bond_usa_10_cr_2' ]
column_o = col_inv1 + col_common1 + col_his1 + col_inv2 + col_common2 + col_his2 + col_cr

col_except_futures = [item for item in column_o if item not in col_futures]

new_col = col_except_futures.copy()

# bank, financeetc는 결측치가 많아서 사용하지 않음.
# df.drop(['bank_1', 'bank_2', 'financeetc_1', 'financeetc_2'], axis=1, inplace=True)   

# col_futures : futures는 당일 종료가 되지 않는 data이므로 제외

In [17]:
# 최초의 empty df 생성
df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')
df_params = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')
iter = 0

In [18]:
#### 신규 분석 데이터는 아래 것으로 사용

com_name = list(code.values())[0][1]

directory_for_ml = '../../data/data_for_ml/predict/'
fname = f'df_{com_name}_combine.pkl'
f_name = directory_for_ml + fname
df_o = pd.read_pickle(f_name) 
df_o = df_o.iloc[:-1] # /predict/를 사용할 경우 마지막 prediction data 제외

In [19]:
len(df_o)

326

In [20]:
train_start_index = list(df_o.index).index(datetime.date(2022, 3, 2))
train_end_index = list(df_o.index).index(datetime.date(2023, 3, 31))
val_end_index = list(df_o.index).index(datetime.date(2023, 4, 28))

In [21]:
df_o_train = df_o.iloc[train_start_index:train_end_index+1]
df_o_val = df_o.iloc[train_end_index+1:val_end_index+1]
df_o_test = df_o.iloc[val_end_index+1:]

In [22]:
df = df_o_train[new_col]
train_input = df.iloc[:, :-5]
train_target = df.iloc[:, -5]

df = df_o_val[new_col]
val_input = df.iloc[:, :-5]
val_target = df.iloc[:, -5]

df = df_o_test[new_col]
test_input = df.iloc[:, :-5]
test_target = df.iloc[:, -5]

In [23]:
scaler = None
scaler = StandardScaler()
scaler.fit(train_input)
train_scaled = scaler.transform(train_input)
val_scaled = scaler.transform(val_input)
test_scaled = scaler.transform(test_input)

In [24]:
'''
# 반복 작업시 여기서 부터 진행 (feature importance로 선정된 새로운 column으로)

df = df_o[new_col]  # 새롭게 선정된 column으로 진행

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

test_interval = int((len(df) - split_n)/3)
data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -5]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -5]
test_input1 = df.iloc[split_n:split_n+test_interval, :-5]
test_input2 = df.iloc[split_n+test_interval: split_n+test_interval*2, :-5]
test_input3 = df.iloc[split_n+test_interval*2:, :-5]
test_target1 = df.iloc[split_n:split_n+test_interval, -5]
test_target2 = df.iloc[split_n+test_interval: split_n+test_interval*2, -5]
test_target3 = df.iloc[split_n+test_interval*2:, -5]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

scaler = None
scaler = StandardScaler()
scaler.fit(train_input)
train_scaled = scaler.transform(train_input)
val_scaled = scaler.transform(val_input)
test_scaled = scaler.transform(test_input)
test_scaled1 = scaler.transform(test_input1)
test_scaled2 = scaler.transform(test_input2)
test_scaled3 = scaler.transform(test_input3)
'''

'\n# 반복 작업시 여기서 부터 진행 (feature importance로 선정된 새로운 column으로)\n\ndf = df_o[new_col]  # 새롭게 선정된 column으로 진행\n\n# train, val,: 8, test: 2\nsplit_ratio = 0.8\nsplit_n = int(len(df)*split_ratio)\n\ntest_interval = int((len(df) - split_n)/3)\ndata = df.iloc[:split_n, :-5]\ntarget = df.iloc[:split_n, -5]\ntest_input = df.iloc[split_n:, :-5]\ntest_target = df.iloc[split_n:, -5]\ntest_input1 = df.iloc[split_n:split_n+test_interval, :-5]\ntest_input2 = df.iloc[split_n+test_interval: split_n+test_interval*2, :-5]\ntest_input3 = df.iloc[split_n+test_interval*2:, :-5]\ntest_target1 = df.iloc[split_n:split_n+test_interval, -5]\ntest_target2 = df.iloc[split_n+test_interval: split_n+test_interval*2, -5]\ntest_target3 = df.iloc[split_n+test_interval*2:, -5]\n\ntrain_input, val_input, train_target, val_target =      train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)\n\nscaler = None\nscaler = StandardScaler()\nscaler.fit(train_input)\ntrain_scaled = scaler.transform(train_in

In [75]:
search_space = {
    'boosting_type' : ['gbdt', 'dart', ],# ['gbdt', 'dart', 'goss'],
    'learning_rate': (0.0001, 0.004, 'log-uniform'),
    'num_leaves': (3, 20),      
    'max_depth': (0, 50), #[-1], #(0, 50),
    'min_child_samples': (5, 40),
#     'min_child_weight': (0.001, 5.0), # 값을 변동시 같은 값이 최적값으로 선정되더라고 precision이 틀림. 반드시 값을 바꾸면서 진행해야 함.
#     'min_split_loss': (0, 5), 
    'max_bin': (100, 1000), # 사용시 0.75%로 올라감 default: 255
#     'max_cat_threshold' : (1, 100),
    'subsample': (0.005, 1.0, 'uniform'), # == bagging_fraction
#     'subsample_freq': (1, 10), # mobis에서 적용시 값이 많이 올라감. default:0
    'colsample_bytree': (0.5, 1.0, 'uniform'),
    'subsample_for_bin': (100000, 1000000),
    'scale_pos_weight' : (0.2, 2.0, 'uniform'),
#     'scale_pos_weight': np.arange(0.2, 2, 0.1) ,
    'n_estimators': (700, 5000),
    'reg_alpha' : (0.0, 3.0, 'uniform'), # =='lambda_l1': [0, 5], # default 0
    'reg_lambda' : (0.0, 3.0, 'uniform'), # == lambda_l2': [0, 5], #default 0    
#     'reg_alpha' : np.arange(0, 4, 0.1),
#     'reg_lambda' : np.arange(0, 4, 0.1),
    'force_col_wise': ['true'], 
    'importance_type': ['split'], # ['gain'], default: split
    'boost_from_average' : [True, False],
    'objective': ['binary'],
    'metric': ['binary_logloss'],
}

params_space = search_space.copy() # for Bayesian Optimization

## BayesSearchCV

In [83]:
param_grid = {
    'cv' : 5,  # usually 5 or 10, default: 3
#     'cv' : None,
#     'scoring' : None,
#     'scoring' : 'f1',
#     'scoring' : 'roc_auc',
    'scoring' : 'precision',
#     'scoring' : 'average_precision',
#     'scoring' : 'accuracy',
    'num_col' : len(new_col),
    'iterations' : 20,  # default : 50, number of parameter settings.
    # The number of parameter settings that are tried is given by n_iter.
             }

In [84]:
# directory_for_model = '../../data/data_for_ml/model/model/0. test/'
directory_for_model = '../../data/data_for_ml/model/model/'

In [85]:
# directory가 없으면 만드는 과정
# if not os.path.exists(com_name):
#     os.makedirs(com_name)
if not os.path.exists(directory_for_model+com_name):
    os.makedirs(directory_for_model+com_name)

iter = iter + 1

lgbm = None
lgbmBS = None
gsbs = 'bs'

lgbmBS = BayesSearchCV( estimator = lightgbm.LGBMClassifier(verbose=0, random_state=42),
                       search_spaces = params_space,
                       scoring = param_grid['scoring'],
                       cv = StratifiedKFold( n_splits=param_grid['cv'],
                                             shuffle=True,
                                             random_state=42 ),
                       n_jobs = -1, # 자동 검색 적용
                       n_iter = param_grid['iterations'],   
                       verbose = 0, refit = True, random_state = 42 
                      )

print("*** after lgbm BS ******")
lgbmBS.fit(train_scaled, train_target, eval_metric = 'logloss') 

# save model
stamp = datetime.datetime.today().isoformat() # 파일명 끝에 생성날짜 시간 추가
dt = re.sub(r'[-:T]', '', stamp[5:16])
dt = f'{dt[:4]}_{dt[4:]}'

df_estimator = make_df_from_estimator(lgbmBS.best_estimator_.get_params(), iter)
df_estimator.sort_index(inplace=True) # alphabet 순으로 보기 편하게

# metrics accuracy,,,, 3단계 precision 등까지. dictionary
result_dict = calc_results(lgbmBS, lgbmBS.best_estimator_, 
                           train_scaled, val_scaled, test_scaled,  
                           train_target, val_target, test_target,
                          )

df_grid = make_df_from_estimator(param_grid, iter) # gridcv parameter
df_grid.sort_index(inplace=True) # alphabet 순으로 보기 편하게
df_result = make_df_from_estimator(result_dict, iter)  # dict 를 df로
df_concat = pd.concat([df_grid, df_estimator, df_result])

df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)

# val_test = df_concat.loc['acc_val'].iloc[0]
# acc_test = df_concat.loc['acc_test'].iloc[0]
# precision = df_concat.loc['test_precision'].iloc[0]
# f1score = df_concat.loc['f1score'].iloc[0]

params_search = add_dict(param_grid, search_space)
df_params_search = make_df_from_estimator(params_search, iter)
df_params = pd.merge(df_params,df_params_search, how='outer', left_index=True, right_index=True)
  
print("******* No.{} BS Process is Done! ********".format(iter))
    
print("**** End of BayesSearchCV Process ****")

*** after lgbm BS ******
******* No.8 BS Process is Done! ********
**** End of BayesSearchCV Process ****


In [86]:
df_one = df_combine_sorted(df_base, df_params)

In [87]:
row_index =  ['train_precision', 'val_precision', 'test_precision']
df_style(df_one.iloc[52:80, :], row_index)

,parameter,iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,iter_8
0,verbose,0,0,0,0,0,0,0,0
1,best_score,0.635111,0.682308,0.630000,0.666667,0.632222,0.625000,0.608333,0.547619
2,best_index,18.000000,63.000000,24.000000,49.000000,14.000000,21.000000,44.000000,15.000000
3,acc_train,0.688716,0.692607,0.587549,0.618677,0.708171,0.630350,0.649805,0.653696
4,acc_val,0.631579,0.631579,0.631579,0.631579,0.578947,0.526316,0.578947,0.578947
5,acc_test,0.611111,0.722222,0.777778,0.722222,0.722222,0.722222,0.722222,0.722222
6,train_precision,0.828125,0.852459,1.000000,1.000000,1.000000,0.846154,0.970588,0.779661
7,train_tn,124.000000,126.000000,135.000000,135.000000,135.000000,129.000000,134.000000,122.000000
8,train_fp,11.000000,9.000000,0.000000,0.000000,0.000000,6.000000,1.000000,13.000000
9,train_fn,69.000000,70.000000,106.000000,98.000000,75.000000,89.000000,89.000000,76.000000


In [89]:
row_index = ['n_estimators', 'reg_alpha', 'max_bin', 'subsample_for_bin']
df_style(df_one.head(60), row_index)

,parameter,iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,iter_8
0,boost_from_average,"[True, False]","[True, False]","[True, False]","[True, False]","[True, False]","[True, False]","[True, False]","[True, False]"
1,boost_from_average,True,True,True,False,True,False,True,False
2,boosting_type,"['gbdt', 'dart']","['gbdt', 'dart']","['gbdt', 'dart']","['gbdt', 'dart']","['gbdt', 'dart']","['gbdt', 'dart']","['gbdt', 'dart']","['gbdt', 'dart']"
3,boosting_type,dart,dart,gbdt,gbdt,dart,gbdt,gbdt,gbdt
4,class_weight,None,None,None,None,None,None,None,None
5,colsample_bytree,"(0.5, 1.0, 'uniform')","(0.5, 1.0, 'uniform')","(0.5, 1.0, 'uniform')","(0.5, 1.0, 'uniform')","(0.5, 1.0, 'uniform')","(0.5, 1.0, 'uniform')","(0.5, 1.0, 'uniform')","(0.5, 1.0, 'uniform')"
6,colsample_bytree,1.000000,1.000000,0.969645,0.859305,0.877137,0.912583,0.890697,1.000000
7,cv,5,5,5,5,5,5,5,5
8,cv,5,5,5,5,5,5,5,5
9,force_col_wise,['true'],['true'],['true'],['true'],['true'],['true'],['true'],['true']


In [90]:
# parameter 조합별 측정치 비교
df_result =  pd.DataFrame(lgbmBS.cv_results_)
df_result.loc[:, ['mean_test_score', 'std_test_score', 'rank_test_score']].sort_values(by='mean_test_score', ascending=False)

,mean_test_score,std_test_score,rank_test_score
15,0.547619,0.054294,1
16,0.544177,0.131561,2
5,0.542063,0.113955,3
11,0.531712,0.092139,4
7,0.528963,0.066626,5
19,0.526740,0.082149,6
8,0.526217,0.057509,7
12,0.525920,0.032392,8
2,0.508408,0.064226,9
13,0.501869,0.016625,10


In [91]:
df_result.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_boost_from_average', 'param_boosting_type',
       'param_colsample_bytree', 'param_force_col_wise',
       'param_importance_type', 'param_learning_rate', 'param_max_bin',
       'param_max_depth', 'param_metric', 'param_min_child_samples',
       'param_n_estimators', 'param_num_leaves', 'param_objective',
       'param_reg_alpha', 'param_reg_lambda', 'param_scale_pos_weight',
       'param_subsample', 'param_subsample_for_bin', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score'],
      dtype='object')

In [60]:
# parameter 조합별 측정치 비교
df_result =  pd.DataFrame(lgbmBS.cv_results_)
df_result.loc[:, ['mean_test_score', 'std_test_score', 'rank_test_score']].sort_values(by='mean_test_score', ascending=False)

,mean_test_score,std_test_score,rank_test_score
49,0.666667,0.278887,1
31,0.648333,0.191079,2
27,0.633333,0.239212,3
25,0.596667,0.246396,4
13,0.575556,0.117252,5
32,0.560476,0.159943,6
11,0.541810,0.067071,7
5,0.536648,0.107596,8
30,0.530980,0.090293,9
42,0.529474,0.086072,10


In [ ]:
# search_space, parameters and results save
df_one.to_csv(f'{directory_for_model}{com_name}/params_results_{gsbs}_df_{dt}.csv')
df_one.to_pickle(f'{directory_for_model}{com_name}/params_results_{gsbs}_df_{dt}.pkl')

In [ ]:

# 아래 내용은 위의 df_one을 대체함.
# parameter and results save
df_base.to_csv(f'{directory_for_model}{com_name}/lgbm_{gsbs}_df_{dt}.csv')
df_base.to_pickle(f'{directory_for_model}{com_name}/lgbm_{gsbs}_df_{dt}.pkl')

# search_space, parameters save
df_params.to_csv(f'{directory_for_model}{com_name}/params_{gsbs}_df_{dt}.csv')
df_params.to_pickle(f'{directory_for_model}{com_name}/params_{gsbs}_df_{dt}.pkl')

In [ ]:

'''
# save model
# 필요시 저장. 가장 좋은 결과만 저장하고자할 때는 밑의 save_best_results만 진행하면 됨.
joblib.dump(lgbmBS.best_estimator_, f'{directory_for_model}{com_name}/estimator_{gsbs}_{dt}_v{iter}.pkl') # bayessearchcv 저장
save_to_pickle(f'{directory_for_model}{com_name}/model_{gsbs}_{dt}_v{iter}_p.pkl', lgbmBS.best_estimator_)
joblib.dump(scaler, f'{directory_for_model}{com_name}/scaler_{gsbs}_{dt}_v{iter}.pkl') # scaler 저장
save_to_pickle(f'{directory_for_model}{com_name}/columns_{gsbs}_{dt}_{len(new_col)}_{round(precision*100):2d}%_{round(f1score*100):2d}%_ver{iter}.pkl', new_col)
save_list_to_csv(f'{directory_for_model}{com_name}/columns_{gsbs}_{dt}_{len(new_col)}_{round(precision*100):2d}%_{round(f1score*100):2d}%_ver{iter}.csv', new_col)
'''

In [ ]:
# 최종 가장 좋은 결과 저장
save_best_results(com_name, lgbmBS.best_estimator_, scaler, new_col, df_base)

In [ ]:
print(lgbmBS.best_estimator_)

In [ ]:
model = lgbmBS.best_estimator_
feature_df = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)

In [ ]:
len(feature_df)

In [ ]:
feature_df.tail(70)

In [ ]:
new_col = list(feature_df.index[:35]) +  ['cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']